### English to Bahasa Translation using SeaLion-v3-9b-IT instruction tuned LLM based on Gemma2
### This is run in Runpod/remote Jupyter environment

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install
!git clone https://huggingface.co/aisingapore/Gemma-SEA-LION-v3-9B-IT LLM_MODELS/Gemma-SEA-LION-v3-9B-IT

In [ ]:
import os
from pathlib import Path

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "workspace/LLM_MODELS/Gemma-SEA-LION-v3-9B-IT" ############## <--- Change this based on platform and models

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
######### set LLM_MODEL to the same path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization

print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

In [ ]:
######### Quick test generation using existing HFBackend

from llm_backends import HFBackend
import os, traceback

local_path = os.environ.get("LOCAL_MODEL_PATH")
print("Using local_path:", local_path)

try:
    ######### create backend that points to the local model path (this uses the existing class)
    backend = HFBackend(local_model_path=local_path, hf_model_id=None, quantize_4bit=(os.environ.get("LLM_LOAD_IN_4BIT","0")=="1"))
    print("HFBackend initialized OK")
    print("Tokenizer pad_token_id:", getattr(backend.tokenizer, "pad_token_id", None))
    print("Model device:", getattr(backend, "device", None))
    ######### test small generation
    prompt = "Translate to formal logic: If it rains, the ground will be wet."
    out = backend.generate(prompt, max_new_tokens=200, temperature=0.0, top_p=1.0, do_sample=False)
    print("=== GENERATION ===")
    print(out[:2000])
except Exception as e:
    print("HFBackend init / generate failed:")
    traceback.print_exc()

print("Finished generating/testing")

In [ ]:
!python translate_dataset.py --dataset_name ProntoQA --split dev --sample-pct 10 --batch_size 1
print("\nFinished translating dataset\n")